<a href="https://colab.research.google.com/github/Tusharprabhu/Song_recommendationKNN/blob/main/newDataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kagglehub[pandas-datasets]  # Note: In Colab, use ! for pip commands

import kagglehub
from kagglehub import KaggleDatasetAdapter
import pandas as pd


In [2]:
file_path = "spotify-2023.csv"  # Adjust if the file name is different

In [7]:
df = kagglehub.load_dataset(
        KaggleDatasetAdapter.PANDAS,
        "nelgiriyewithana/top-spotify-songs-2023",
        file_path,
        pandas_kwargs={
            "low_memory": False,
            "encoding": "latin1"  # Try 'latin1' to handle special characters
        }
    )
print("Dataset loaded successfully.")

<ipython-input-7-824f33057944>:1: DeprecationWarning: load_dataset is deprecated and will be removed in future version.
  df = kagglehub.load_dataset(


Dataset loaded successfully.


In [8]:
print("Shape of the dataset:", df.shape)
print("Columns in the dataset:", df.columns.tolist())



Shape of the dataset: (953, 24)
Columns in the dataset: ['track_name', 'artist(s)_name', 'artist_count', 'released_year', 'released_month', 'released_day', 'in_spotify_playlists', 'in_spotify_charts', 'streams', 'in_apple_playlists', 'in_apple_charts', 'in_deezer_playlists', 'in_deezer_charts', 'in_shazam_charts', 'bpm', 'key', 'mode', 'danceability_%', 'valence_%', 'energy_%', 'acousticness_%', 'instrumentalness_%', 'liveness_%', 'speechiness_%']


In [9]:
import pandas as pd

# Print basic info about the dataset
print("Dataset Info:")
print(df.info())  # Shows data types and non-null counts

# Check for missing values
print("\nMissing Values:")
print(df.isnull().sum())

# Optional: Show first 5 rows to understand the data
print("\nFirst 5 Rows:")
print(df.head())

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 953 entries, 0 to 952
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   track_name            953 non-null    object
 1   artist(s)_name        953 non-null    object
 2   artist_count          953 non-null    int64 
 3   released_year         953 non-null    int64 
 4   released_month        953 non-null    int64 
 5   released_day          953 non-null    int64 
 6   in_spotify_playlists  953 non-null    int64 
 7   in_spotify_charts     953 non-null    int64 
 8   streams               953 non-null    object
 9   in_apple_playlists    953 non-null    int64 
 10  in_apple_charts       953 non-null    int64 
 11  in_deezer_playlists   953 non-null    object
 12  in_deezer_charts      953 non-null    int64 
 13  in_shazam_charts      903 non-null    object
 14  bpm                   953 non-null    int64 
 15  key                   858 

In [10]:
df['key'].fillna(df['key'].mode()[0], inplace=True)  # Fill missing 'key' with most common value
df['in_shazam_charts'].fillna(0, inplace=True)  # Fill missing 'in_shazam_charts' with 0

# Convert 'streams' to numeric, handle errors
df['streams'] = pd.to_numeric(df['streams'], errors='coerce')
df.dropna(subset=['streams'], inplace=True)  # Drop rows where 'streams' is invalid

# Verify changes
print("Missing Values After Cleaning:")
print(df.isnull().sum())
print("\nData Types:")
print(df.dtypes)

Missing Values After Cleaning:
track_name              0
artist(s)_name          0
artist_count            0
released_year           0
released_month          0
released_day            0
in_spotify_playlists    0
in_spotify_charts       0
streams                 0
in_apple_playlists      0
in_apple_charts         0
in_deezer_playlists     0
in_deezer_charts        0
in_shazam_charts        0
bpm                     0
key                     0
mode                    0
danceability_%          0
valence_%               0
energy_%                0
acousticness_%          0
instrumentalness_%      0
liveness_%              0
speechiness_%           0
dtype: int64

Data Types:
track_name               object
artist(s)_name           object
artist_count              int64
released_year             int64
released_month            int64
released_day              int64
in_spotify_playlists      int64
in_spotify_charts         int64
streams                 float64
in_apple_playlists        int64

<ipython-input-10-7348fd6fb380>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['key'].fillna(df['key'].mode()[0], inplace=True)  # Fill missing 'key' with most common value
<ipython-input-10-7348fd6fb380>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing

In [23]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors

# Select numerical features for KNN
numerical_features = [
    'danceability_%','energy_%'
]

# Create a new DataFrame with only numerical features
df_features = df[numerical_features]

# Standardize the features
scaler = StandardScaler()
df_scaled = pd.DataFrame(
    scaler.fit_transform(df_features),
    columns=numerical_features,
    index=df.index  # Preserve index for alignment
)

# Initialize and train the KNN model
knn = NearestNeighbors(n_neighbors=6, algorithm='auto', metric='euclidean')
knn.fit(df_scaled)

print("KNN model trained successfully with features:", numerical_features)

KNN model trained successfully with features: ['danceability_%', 'energy_%']


In [24]:
def recommend_songs(song_name: str, df: pd.DataFrame, df_scaled: pd.DataFrame, knn: NearestNeighbors, top_n: int = 5):

    # Find the song (case-insensitive)
    song_indices = df[df['track_name'].str.lower() == song_name.lower()].index

    if len(song_indices) == 0:
        return "Song not found in the dataset. Please check the name and try again."

    # Get the first matching song's index
    song_index = song_indices[0]

    # Get scaled features for the song
    song_features = df_scaled.iloc[song_index].values.reshape(1, -1)

    # Find nearest neighbors
    distances, indices = knn.kneighbors(song_features)

    # Get indices of recommended songs (exclude the input song)
    recommended_indices = indices[0][1:top_n+1]

    # Select columns to return: track_name, artist(s)_name, and KNN features
    output_columns = ['track_name', 'artist(s)_name'] + [
        'danceability_%', 'energy_%','valence_%' ]

    # Return recommended songs
    recommendations = df.iloc[recommended_indices][output_columns]
    # Note: 'genres' not included as the dataset lacks a genres column

    print(f"Recommendations for '{song_name}':")
    return recommendations

# Example usage
if __name__ == "__main__":
    song_input = "Blinding Lights"  # Replace with a song from your dataset
    recommendations = recommend_songs(song_input, df, df_scaled, knn)
    print(recommendations)

Recommendations for 'Blinding Lights':
                       track_name            artist(s)_name  danceability_%  \
395                    Space Song               Beach House              51   
450           Underneath the Tree            Kelly Clarkson              51   
772  Cooped Up (with Roddy Ricch)  Post Malone, Roddy Ricch              50   
451                 Feliz Navidad           Josï¿½ï¿½ Felic              50   
304               SORRY NOT SORRY        Tyler, The Creator              48   

     energy_%  valence_%  
395        79         62  
450        81         69  
772        78         39  
451        82         96  
304        80         50  


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


In [25]:
song_input = "Feliz Navidad"  # Replace with a song from your dataset
recommendations = recommend_songs(song_input, df, df_scaled, knn)
print(recommendations)

Recommendations for 'Feliz Navidad':
              track_name       artist(s)_name  danceability_%  energy_%  \
793    House Of Memories  Panic! At The Disco              51        82   
450  Underneath the Tree       Kelly Clarkson              51        81   
55       Blinding Lights           The Weeknd              50        80   
768      United In Grief       Kendrick Lamar              52        83   
304      SORRY NOT SORRY   Tyler, The Creator              48        80   

     valence_%  
793         48  
450         69  
55          38  
768         32  
304         50  


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
